In [2]:
#Random Forest algorithm
#replicate findings from Walters et al 2014

In [3]:
import io, os, sys, types
from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell

import biom
import sklearn
import pandas as pd

In [4]:
#Amish dataset
text_file = "/Users/shwetakinger/Desktop/ERSP_data/merged_bmi_mapping_final__original_study_amish_Fraser__.txt"
biom_file = "/Users/shwetakinger/Downloads/per_study_otu_tables/filtered_otu_table__original_study_amish_Fraser__.biom"

In [5]:
m = pd.read_csv(text_file, sep="\t", index_col=0)

In [6]:
m.columns

Index(['BarcodeSequence', 'LinkerPrimerSequence', 'BMI', 'bmi_group_binned',
       'bmi_group_coded', 'original_study', 'combined_study_bmi_group',
       'PCR_PRIMERS', 'TARGET_SUBFRAGMENT', 'AGE', 'ELEVATION', 'LONGITUDE',
       'COUNTRY', 'SEQUENCING_METH', 'SAMPLE_CENTER', 'Description_duplicate',
       'ReversePrimer', 'COLLECTION_DATE', 'SEX', 'FAMILY_RELATIONSHIP_GG',
       'STUDY_CENTER', 'EXPERIMENT_CENTER', 'bmi_group_amish', 'RUN_CENTER',
       'LATITUDE', 'Description'],
      dtype='object')

In [7]:
m.index


Int64Index([  1,   6,  16,  18,  32,  35,  53,  54,  60,  64,
            ...
            471, 473, 475, 476, 481, 482, 488, 595, 598, 599],
           dtype='int64', name='#SampleID', length=325)

In [8]:
b= biom.load_table(biom_file)


In [9]:
#select only samples that fit LEAN OR OBESE
i = m[m.bmi_group_coded != "Overweight"].index
print(i[0])
i_2 = m[m.bmi_group_coded != "Overweight"]

i_2.loc[i_2.bmi_group_binned == 3, 'bmi_group_binned'] = 0
#print(i_2)

1


/Users/shwetakinger/anaconda/lib/python3.5/site-packages/pandas/core/indexing.py:465: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [10]:
writer = pd.ExcelWriter('biom_original_1.xlsx', engine='xlsxwriter')



In [11]:
p = pd.DataFrame(b.matrix_data.T.todense().astype(int), index=b.ids(axis="sample"), columns=b.ids(axis="observation"))
print(p)

     304309  193008  190299  191393  193061  186596  174924  296166  177792  \
274       1       0       0       0       0       0       0       0      45   
35        0       1       0       0       0       0       2       0       0   
360       0       1       0       0       0       0       0       0       0   
347       0       1       2       0       0       0       0       0       0   
349       0       2       1       0       0       0       0       0       0   
329       0       0       2       0       0       0       0       0      10   
336       0       0       3       0       0       0       1       0       0   
132       0       0       2       0       0       0       0       0      29   
171       0       0       1       0       0       0       0       0       0   
148       0       0       1       0       0       0       0       0       1   
443       0       0       1       0       0       1       0       0       2   
435       0       0       1       0       0       0 

In [12]:
# Convert the dataframe to an XlsxWriter Excel object.
p.to_excel(writer, sheet_name='Sheet1')

# Close the Pandas Excel writer and output the Excel file.
writer.save()

In [13]:
# load sorted file
sorted_m = pd.read_csv("/Users/shwetakinger/biom_original_1.csv", index_col=0)
print(sorted_m)

     304309  193008  190299  191393  193061  186596  174924  296166  177792  \
1         0       0       0       0       0       0       0       0       0   
2         0       0       0       0       0       0       0       0       1   
3         0       0       0       0       0       0       0       0       4   
5         0       0       0       0       0       0       0       0       0   
6         0       0       0       0       0       0       0       1       0   
7         0       0       0       0       0       0       0       0       0   
8         0       0       0       0       0       1       0       0       0   
9         0       0       0       0       0       0       0       0       0   
11        0       0       0       0       0       0       0       0       0   
12        0       0       0       1       0       0       1       0       1   
13        0       0       0       0       0       0       0       0       0   
14        0       0       0       0       0       0 

In [14]:
#sorted_m.index.get_loc(6)
#sorted_m.iloc[[4]]

data = pd.DataFrame([])
for n in range(0, 222):
    #print(n)
    num = sorted_m.index.get_loc(i[n])
    data = data.append(sorted_m.iloc[[num]])
    
print(data)


     304309  193008  190299  191393  193061  186596  174924  296166  177792  \
1         0       0       0       0       0       0       0       0       0   
6         0       0       0       0       0       0       0       1       0   
16        0       0       0       4       0       0       0       0       2   
18        0       0       0       0       0       0       0       0       0   
32        0       0       0       0       0       0       1       0      11   
35        0       1       0       0       0       0       2       0       0   
53        0       0       0       0       0       0       0       0       1   
54        0       0       0       0       0       1       1       0       3   
60        0       0       0       0       0       0       3       0       0   
64        0       0       0       0       0       0       0       0       0   
81        0       0       0       0       0       0       0       0       1   
97        0       0       0       0       0       0 

In [15]:
from sklearn import model_selection
from sklearn.ensemble import RandomForestClassifier


In [16]:
from sklearn.cross_validation import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_predict
iter_range = range(1, 6)
# empty list to store scores
rForest_scores = []
average_accuracy = 0.0;

for i in iter_range:
    classifier = RandomForestClassifier(n_estimators=1000)
    #cross-validation score
    #scores = cross_val_score(classifier, c.fillna(0).as_matrix(), i_2.bmi_group_coded, cv=10, scoring='accuracy')
    predicted = cross_val_predict(classifier, data, i_2.bmi_group_binned, cv=10)
    #cross-validation AUC score
    #scores = cross_val_score(classifier, c.fillna(0).as_matrix(), i_2.bmi_group_binned, cv=10, scoring='roc_auc')
    #rForest_scores.append(scores.mean())
    #average_accuracy+=scores.mean()
    print(confusion_matrix(i_2.bmi_group_binned, predicted))

#print(rForest_scores)
#print(average_accuracy/(5.0))

[[144   2]
 [ 74   2]]
[[145   1]
 [ 72   4]]
[[141   5]
 [ 73   3]]
[[143   3]
 [ 74   2]]
[[145   1]
 [ 73   3]]


In [17]:
from sklearn.cross_validation import cross_val_score
iter_range = range(1, 6)
# empty list to store scores
rForest_scores = []
average_accuracy = 0.0;

for i in iter_range:
    classifier = RandomForestClassifier(n_estimators=1000)
    #cross-validation score
    #scores = cross_val_score(classifier, c.fillna(0).as_matrix(), i_2.bmi_group_coded, cv=10, scoring='accuracy')
    #cross-validation AUC score
    scores = cross_val_score(classifier, data, i_2.bmi_group_binned, cv=10, scoring='roc_auc')
    rForest_scores.append(scores.mean())
    average_accuracy+=scores.mean()

print(rForest_scores)
print(average_accuracy/(5.0))

[0.64705782312925175, 0.66152210884353735, 0.66184523809523799, 0.63136054421768706, 0.67398809523809522]
0.655154761905
